### Query 3

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, when, expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, ArrayType
from pyspark.sql.functions import col, year, to_timestamp
from pyspark.sql.functions import broadcast
from sedona.spark import *
import time

# Δημιουργούμε SparkSession

spark = SparkSession.builder.appName("ZipCodeAnalysis").config("spark.sql.debug.maxToStringFields", 1000).getOrCreate()
sedona = SedonaContext.create(spark)
SedonaRegistrator.registerAll(spark)
spark.sparkContext.setLogLevel("DEBUG")

start_time = time.time()


CrimeDataFile = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
CensusBlockFile = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
IncomeFile = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv"

crimeSchema = StructType([
    StructField("DR_NO", StringType(), True),
    StructField("Date Rptd", StringType(), True),
    StructField("DATE OCC", StringType(), True),
    StructField("TIME OCC", StringType(), True),
    StructField("AREA", StringType(), True),
    StructField("AREA NAME", StringType(), True),
    StructField("Rpt Dist No", StringType(), True),
    StructField("Part 1-2", StringType(), True),
    StructField("Crm Cd", StringType(), True),
    StructField("Crm Cd Desc", StringType(), True),
    StructField("Mocodes", StringType(), True),
    StructField("Vict Age", StringType(), True),
    StructField("Vict Sex", StringType(), True),
    StructField("Vict Descent", StringType(), True),
    StructField("Premis Cd", StringType(), True),
    StructField("Premis Desc", StringType(), True),
    StructField("Weapon Used Cd", StringType(), True),
    StructField("Weapon Desc", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Status Desc", StringType(), True),
    StructField("Crm Cd 1", StringType(), True),
    StructField("Crm Cd 2", StringType(), True),
    StructField("Crm Cd 3", StringType(), True),
    StructField("Crm Cd 4", StringType(), True),
    StructField("LOCATION", StringType(), True),
    StructField("Cross Street", StringType(), True),
    StructField("LAT", StringType(), True),
    StructField("LON", StringType(), True)
])



incomeSchema = StructType([
    StructField("Zip Code", StringType(), True),
    StructField("Community", StringType(), True),
    StructField("Estimated Median Income", StringType(), True)
])

#Φορτώνουμε τα DataSets

CrimeDataFrame = spark.read.csv(CrimeDataFile, header=True, schema=crimeSchema)

#Κρατάμε μόνο τα εγκλήματα που έγιναν το 2010
CrimeDataFrame = CrimeDataFrame \
    .withColumn("parsed_date", to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")) \
    .filter(year(col("parsed_date")) == 2010) \
    .drop("parsed_date")  

IncomeDataFrame = spark.read.csv(IncomeFile, header=True, schema = incomeSchema)



#Για το blocks χρησιμοποιούμε το Sedona και το κάνουμε flatten

blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(CensusBlockFile) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

BlocksDataFrame = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

#Κρατάμε μόνο τα blocks της πόλης του Los Angeles
BlocksDataFrame = BlocksDataFrame.filter(col("CITY") == "Los Angeles")




#Φτιάχνουμε Point Geometry για κάθε έγκλημα

CrimeDataFrame = CrimeDataFrame.filter(col("LAT").isNotNull() & col("LON").isNotNull())
CrimeDataFrame = CrimeDataFrame.withColumn(
    "crime_geometry",
    expr("ST_Point(CAST(LON AS DOUBLE), CAST(LAT AS DOUBLE))")
).withColumn("crime_geometry", col("crime_geometry").cast(GeometryType()))



#Μετατρέπουμε το geometry του κάθε block σε κατάλληλο format για το sedona

BlocksDataFrame = BlocksDataFrame.withColumn(
    "geometry",
    col("geometry").cast(GeometryType())
)


# Repartition crimes and blocks for better parallelism
#CrimeDataFrame = CrimeDataFrame.repartition(200, "LON", "LAT")  # Example partitioning by longitude and latitude
#BlocksDataFrame = BlocksDataFrame.repartition(50, "geometry")     # Partitioning by geometry if available

#Για κάθε block, βρίσκουμε πόσα εγκλήματα έγιναν σε αυτό χρησιμοποιώντας το ST_contains του Sedona

spatial_join_df = (
    CrimeDataFrame.join(
        BlocksDataFrame.hint("shuffle_replicate_nl"),
        expr("ST_Contains(geometry, crime_geometry)"),  # Use Sedona spatial function
    )
    .select("geometry", "DR_NO")  # Select required columns
    .groupBy("geometry")  # Group by blocks' geometry
    .agg(expr("COUNT(*)").alias("crimes_committed"))  # Count crimes for each block
)


#Προσθέτουμε στο Blocks dataset μία στήλη με τα εκλήματα που έγιναν σε κάθε block
BlocksDataFrame = (
    BlocksDataFrame.join(spatial_join_df, BlocksDataFrame.geometry == spatial_join_df.geometry, "left")
    .drop("geometry")  
    .fillna({"crimes_committed": 0})  
)




#Συνεχίζουμε γκρουπάροντας τα blocks που έχουν κοινό zip code ώστε να βγάλουμε συνολικ΄ό housing και population για κάθε zip code


aggregatedBlocksDataFrame = (
     BlocksDataFrame
    .groupBy("ZCTA10","COMM")
    .agg(
        sum(col("HOUSING10")).alias("total_housing"),
        sum(col("POP_2010")).alias("total_population"),
        sum(col("crimes_committed")).alias("total_crimes")
    )
)

#Για το Income φτιάχνουμε τη μορφή του ώστε να μπορεί να γίνει double για πράξεις και στα blocks δίνουμε στον ταχυδρομικό
#κώδικα το όνομα "Zip Code" ώστε να είναι ίδιο με του LA_Income

aggregatedBlocksDataFrame = aggregatedBlocksDataFrame.withColumnRenamed("ZCTA10", "Zip Code")

IncomeDataFrame = IncomeDataFrame.withColumn(
    "Estimated Median Income",
    regexp_replace(col("Estimated Median Income"), "[$,]", "")
)

#Κάνουμε Join το blocks dataset με το income dataset πάνω στο κοινό Zip Code ώστε να πάρουμε μέσο εισόδημα ανά σπίτι για κάθε zip code

CombinedDataFrame = (
    aggregatedBlocksDataFrame
    .join(broadcast(IncomeDataFrame), on="Zip Code", how="inner")
    .withColumn("Estimated Median Income", col("Estimated Median Income").cast("double"))
    .withColumn("total_housing", col("total_housing").cast("double"))
    .withColumn("total_population", col("total_population").cast("double"))
    .withColumn("total_crimes", col("total_crimes").cast("double"))
)

CombinedDataFrame.explain(mode="formatted")

# Για κάθε Zip Code πολλαπλασιάζουμε σπίτια επί μέσο εισόδημα ανά σπίτι για να βρούμε το συνολικό εισόδημα 

CombinedDataFrame = CombinedDataFrame.withColumn(
    "income_per_zip",
    col("total_housing") * col("Estimated Median Income")  
)

#Επειδή κάθε Comm έχει πολλά Zip Codes , βρίσκουμε για κάθε Comm το άθροισμα
#του εισοδήματος, του πληθυσμού και των εγκλημάτων των Zip Codes του


CommAggregatesDF = (
    CombinedDataFrame
    .groupBy("COMM")
    .agg(
        sum("income_per_zip").alias("total_income_in_comm"),
        sum("total_population").alias("total_population_in_comm"),
        sum("total_crimes").alias("total_crimes_in_comm")
    )
)

#Για κάθε Comm κάνουμε τις διαιρέσεις με το πληθυσμό για να βρούμε
#εισόδημα ανά άτομο και εγκλήματα ανά άτομο

CommFinalDF = CommAggregatesDF.withColumn(
    "income_per_person",
    col("total_income_in_comm") / col("total_population_in_comm")
).withColumn(
    "crimes_per_person",
    col("total_crimes_in_comm") / col("total_population_in_comm")
)

CommFinalDF = CommFinalDF.orderBy(col("income_per_person").asc())

#Εκτυπώνουμε τα αποτελέσματα

CommResults = CommFinalDF.collect()

# Define the header and a separator
header = f"{'Comm':<40} {'Income Per Person':<25} {'Crimes Per Person':<25}"
separator = "-" * len(header)

# Print the header
print(header)
print(separator)

# Iterate and print rows
for row in CommResults:
    comm = row['COMM']
    income_per_person = row['income_per_person']
    crimes_per_person = row['crimes_per_person']
    if income_per_person is not None and crimes_per_person is not None:
        print(f"{comm:<40} {income_per_person:<25.2f} {crimes_per_person:<25.4f}")
    else:
        print(f"{comm:<40} {'0':<40} {'0':<40}")


        

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\n\nTime taken: {elapsed_time:.2f} seconds")

